In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import dcor as dc
from scipy.cluster.hierarchy import linkage, cut_tree, dendrogram
from scipy.spatial.distance import squareform
from scipy.stats import norm
from math import sqrt,  tanh, ceil, log, cos, pi, sin


In [ ]:
import os
original_dir = os.getcwd()
os.chdir('..\\src\\robustOptimPack\\wrapping')
from wrapping_funcs import *
os.chdir(original_dir)

In [ ]:
def plot_dendrogram(model, **kwargs):

    # Children of hierarchical clustering
    children = model.children_

    # Distances between each pair of children
    # Since we don't have this information, we can use a uniform one for plotting
    distance = np.arange(children.shape[0])

    # The number of observations contained in each cluster level
    no_of_observations = np.arange(2, children.shape[0]+2)

    # Create linkage matrix and then plot the dendrogram
    linkage_matrix = np.column_stack([children, distance, no_of_observations]).astype(float)

    # Plot the corresponding dendrogram
    dendrogram(linkage_matrix, **kwargs)


In [ ]:
n_reps = 100
n_obs_vec = [50,100,500,1000]
res_mat_noout= np.ones((len(n_obs_vec),n_reps, 4 ))
mean_vec = [0, 0,0]
cov_mat = [[1, 0,0], [0, 1,0], [0,0,1]]  # diagonal covariance
sd_white_noise = sqrt(0.01)
n_clusts = 3
link_method = 'average'
out_fraction = 0.2
out_dist = 6



#### Basic simulation with no outliers

In [ ]:
 for cur_nobs in range(len(n_obs_vec)): 
     n_obs = n_obs_vec[cur_nobs]
     #print("current n_obs is ", n_obs)
     
     for cur_rep in range(n_reps):
        
         np.random.seed(cur_rep)
         #print('current rep', cur_rep+1)
         
         sd_white_noise = sqrt(0.01)
        
         
         x_1 = np.random.uniform(0,1,n_obs)
         x_1 = -1 + x_1*2
        
         u =   np.random.uniform(0,1,n_obs)
         v =   np.random.uniform(0,1,n_obs) 
         x_4 = np.sqrt(-2 *np.log(u) ) *np.cos(2*pi*v)
         x_7 = np.sqrt(-2 *np.log(u) ) *np.sin(2*pi*v)
         
         x_2 = np.tanh(x_1) + np.power(x_1,2)
         x_3 = 2*np.sin(np.abs(x_1)) + np.random.normal(scale = sd_white_noise,size = n_obs)
         
         x_5 = np.sin(x_4) + np.tanh(x_4) + np.random.normal(scale = sd_white_noise,size = n_obs)
         x_6 = np.power(x_4,2) + np.random.normal(scale = sd_white_noise,size = n_obs)
         
         x_8 = np.abs(x_7) + np.random.normal(scale = sd_white_noise,size = n_obs)
         x_9 = np.sin(np.abs(x_7)) + np.random.normal(scale = sd_white_noise,size = n_obs)
         
         dat_sim = np.vstack((x_1,x_2, x_3, x_4, x_5, x_6, x_7, x_8, x_9)).T
         
         #correlation based clustering
         dat_sim_cor = np.corrcoef(dat_sim.T)
         dat_sim_cor_diss = squareform(np.round((1- dat_sim_cor)/2,6))
         cor_clust = linkage(dat_sim_cor_diss, method =link_method )
         cor_clusts = cut_tree(cor_clust, n_clusters= n_clusts)
         res_mat_noout[cur_nobs][cur_rep][0] = np.mean(cor_clusts.flatten() == np.repeat([0,1,2],[3,3,3]))
         
         #dcor based clustering
         dat_sim_dcor = pairwise_dcor(dat_sim)
         dat_sim_dcor_diss = squareform(np.round((1- dat_sim_dcor)/2,6))
         dcor_clust = linkage(dat_sim_dcor_diss, method = link_method)
         dcor_clusts = cut_tree(dcor_clust, n_clusters= n_clusts)
         res_mat_noout[cur_nobs][cur_rep][1] = np.mean(dcor_clusts.flatten() == np.repeat([0,1,2],[3,3,3]))
         
         #w_cor based clustering 
         dat_sim_w_cor = wrapped_covariance_correlation(dat_sim)[1]
         dat_sim_w_cor_diss = squareform(np.round((1- dat_sim_w_cor)/2,6))
         w_cor_clust = linkage(dat_sim_w_cor_diss, method =link_method )
         w_cor_clusts = cut_tree(w_cor_clust, n_clusters= n_clusts)
         res_mat_noout[cur_nobs][cur_rep][2] = np.mean(w_cor_clusts.flatten() == np.repeat([0,1,2],[3,3,3]))

         #w_dcor based clustering 
         dat_sim_w_dcor = wrapped_dcor(dat_sim)
         dat_sim_w_dcor_diss = squareform(np.round((1- dat_sim_w_dcor)/2,6))
         w_dcor_clust = linkage(dat_sim_w_dcor_diss, method =link_method )
         w_dcor_clusts = cut_tree(w_dcor_clust, n_clusters= n_clusts)
         res_mat_noout[cur_nobs][cur_rep][3] = np.mean(w_dcor_clusts.flatten() == np.repeat([0,1,2],[3,3,3]))
         
         

         
         
         
         
         
         
         
         
        
        
         
         

In [ ]:
print( "n = 50: ", np.mean(res_mat_noout[0], axis = 0))
print( "n = 100: ", np.mean(res_mat_noout[1], axis = 0))
print( "n = 500: ", np.mean(res_mat_noout[2], axis = 0))
print( "n = 1000: ", np.mean(res_mat_noout[3], axis = 0))

In [ ]:
sns.pairplot(pd.DataFrame(dat_sim))

#### Simulations with outliers in 3 variables 

In [ ]:
out_dim = 3
res_mat_out_dim3_dist6= np.ones((len(n_obs_vec),n_reps, 4 ))

In [ ]:
 for cur_nobs in range(len(n_obs_vec)): 
     n_obs = n_obs_vec[cur_nobs]
     #print("current n_obs is ", n_obs)
     
     for cur_rep in range(n_reps):
        
         np.random.seed(cur_rep)
         #print('current rep', cur_rep+1)
         
         sd_white_noise = sqrt(0.01)
        
         
         x_1 = np.random.uniform(0,1,n_obs)
         x_1 = -1 + x_1*2
        
         u =   np.random.uniform(0,1,n_obs)
         v =   np.random.uniform(0,1,n_obs) 
         x_4 = np.sqrt(-2 *np.log(u) ) *np.cos(2*pi*v)
         x_7 = np.sqrt(-2 *np.log(u) ) *np.sin(2*pi*v)
         
         x_2 = np.tanh(x_1) + np.power(x_1,2)
         # add outliers to x_2
         out_ind_x_2 = np.random.randint(n_obs, size= round(out_fraction *n_obs) )
         x_2[   out_ind_x_2] = (np.max(x_2) *out_dist)/sqrt(out_dim)
         x_3 = 2*np.sin(np.abs(x_1)) + np.random.normal(scale = sd_white_noise,size = n_obs)
         
         # add outliers to x_5
         x_5 = np.sin(x_4) + np.tanh(x_4) + np.random.normal(scale = sd_white_noise,size = n_obs)
         out_ind_x_5 = np.random.randint(n_obs, size= round(out_fraction *n_obs) )
         x_5[   out_ind_x_5] = (np.max(x_5) *out_dist)/sqrt(out_dim)
         x_6 = np.power(x_4,2) + np.random.normal(scale = sd_white_noise,size = n_obs)
         
         # add outliers to x_8
         x_8 = np.abs(x_7) + np.random.normal(scale = sd_white_noise,size = n_obs)
         out_ind_x_8 = np.random.randint(n_obs, size= round(out_fraction *n_obs) )
         x_8[out_ind_x_8] = (np.max(x_8) *out_dist)/sqrt(out_dim)
         
         x_9 = np.sin(np.abs(x_7)) + np.random.normal(scale = sd_white_noise,size = n_obs)
         
         dat_sim = np.vstack((x_1,x_2, x_3, x_4, x_5, x_6, x_7, x_8, x_9)).T
         
         #correlation based clustering
         dat_sim_cor = np.corrcoef(dat_sim.T)
         dat_sim_cor_diss = squareform(np.round((1- dat_sim_cor)/2,6))
         cor_clust = linkage(dat_sim_cor_diss, method =link_method )
         cor_clusts = cut_tree(cor_clust, n_clusters= n_clusts)
         res_mat_out_dim3_dist6[cur_nobs][cur_rep][0] = np.mean(cor_clusts.flatten() == np.repeat([0,1,2],[3,3,3]))
         
         #dcor based clustering
         dat_sim_dcor = pairwise_dcor(dat_sim)
         dat_sim_dcor_diss = squareform(np.round((1- dat_sim_dcor)/2,6))
         dcor_clust = linkage(dat_sim_dcor_diss, method = link_method)
         dcor_clusts = cut_tree(dcor_clust, n_clusters= n_clusts)
         res_mat_out_dim3_dist6[cur_nobs][cur_rep][1] = np.mean(dcor_clusts.flatten() == np.repeat([0,1,2],[3,3,3]))
         
         #w_cor based clustering 
         dat_sim_w_cor = wrapped_covariance_correlation(dat_sim)[1]
         dat_sim_w_cor_diss = squareform(np.round((1- dat_sim_w_cor)/2,6))
         w_cor_clust = linkage(dat_sim_w_cor_diss, method =link_method )
         w_cor_clusts = cut_tree(w_cor_clust, n_clusters= n_clusts)
         res_mat_out_dim3_dist6[cur_nobs][cur_rep][2] = np.mean(w_cor_clusts.flatten() == np.repeat([0,1,2],[3,3,3]))

         #w_dcor based clustering 
         dat_sim_w_dcor = wrapped_dcor(dat_sim)
         dat_sim_w_dcor_diss = squareform(np.round((1- dat_sim_w_dcor)/2,6))
         w_dcor_clust = linkage(dat_sim_w_dcor_diss, method =link_method )
         w_dcor_clusts = cut_tree(w_dcor_clust, n_clusters= n_clusts)
         res_mat_out_dim3_dist6[cur_nobs][cur_rep][3] = np.mean(w_dcor_clusts.flatten() == np.repeat([0,1,2],[3,3,3]))


In [ ]:
print( "n = 50: ", np.mean(res_mat_out_dim3_dist6[0], axis = 0))
print( "n = 100: ", np.mean(res_mat_out_dim3_dist6[1], axis = 0))
print( "n = 500: ", np.mean(res_mat_out_dim3_dist6[2], axis = 0))
print( "n = 1000: ", np.mean(res_mat_out_dim3_dist6[3], axis = 0))

#### Simulation with outliers in 6 variables 

In [ ]:
out_dim = 6
res_mat_out_dim6_dist6= np.ones((len(n_obs_vec),n_reps, 4 ))

In [ ]:
 for cur_nobs in range(len(n_obs_vec)): 
     n_obs = n_obs_vec[cur_nobs]
     #print("current n_obs is ", n_obs)
     for cur_rep in range(n_reps):
         np.random.seed(cur_rep)
         #print('current rep', cur_rep+1)
         
         sd_white_noise = sqrt(0.01)
        
         
         x_1 = np.random.uniform(0,1,n_obs)
         x_1 = -1 + x_1*2
        
         u =   np.random.uniform(0,1,n_obs)
         v =   np.random.uniform(0,1,n_obs) 
         x_4 = np.sqrt(-2 *np.log(u) ) *np.cos(2*pi*v)
         x_7 = np.sqrt(-2 *np.log(u) ) *np.sin(2*pi*v)
         
         x_2 = np.tanh(x_1) + np.power(x_1,2)
         # add outliers to x_2
         out_ind_x_2 = np.random.randint(n_obs, size= round(out_fraction *n_obs) )
         x_2[   out_ind_x_2] = (np.max(x_2) *out_dist)/sqrt(out_dim)
          # add outliers to x_3
         x_3 = 2*np.sin(np.abs(x_1)) + np.random.normal(scale = sd_white_noise,size = n_obs)
         out_ind_x_3 = np.random.randint(n_obs, size= round(out_fraction *n_obs) )
         x_3[out_ind_x_3] = (np.max(x_3) *out_dist)/sqrt(out_dim)
         
         # add outliers to x_5
         x_5 = np.sin(x_4) + np.tanh(x_4) + np.random.normal(scale = sd_white_noise,size = n_obs)
         out_ind_x_5 = np.random.randint(n_obs, size= round(out_fraction *n_obs) )
         x_5[   out_ind_x_5] = (np.max(x_5) *out_dist)/sqrt(out_dim)
         # add outliers to x_6
         x_6 = np.power(x_4,2) + np.random.normal(scale = sd_white_noise,size = n_obs)
         out_ind_x_6 = np.random.randint(n_obs, size= round(out_fraction *n_obs) )
         x_6[out_ind_x_6] = (np.max(x_6) *out_dist)/sqrt(out_dim)
         
         # add outliers to x_8
         x_8 = np.abs(x_7) + np.random.normal(scale = sd_white_noise,size = n_obs)
         out_ind_x_8 = np.random.randint(n_obs, size= round(out_fraction *n_obs) )
         x_8[out_ind_x_8] = (np.max(x_8) *out_dist)/sqrt(out_dim)
         
          # add outliers to x_9
         x_9 = np.sin(np.abs(x_7)) + np.random.normal(scale = sd_white_noise,size = n_obs)
         out_ind_x_9 = np.random.randint(n_obs, size= round(out_fraction *n_obs) )
         x_9[out_ind_x_9] = (np.max(x_9) *out_dist)/sqrt(out_dim)
         
         dat_sim = np.vstack((x_1,x_2, x_3, x_4, x_5, x_6, x_7, x_8, x_9)).T
         
         #correlation based clustering
         dat_sim_cor = np.corrcoef(dat_sim.T)
         dat_sim_cor_diss = squareform(np.round((1- dat_sim_cor)/2,6))
         cor_clust = linkage(dat_sim_cor_diss, method =link_method )
         cor_clusts = cut_tree(cor_clust, n_clusters= n_clusts)
         res_mat_out_dim6_dist6[cur_nobs][cur_rep][0] = np.mean(cor_clusts.flatten() == np.repeat([0,1,2],[3,3,3]))
         
         #dcor based clustering
         dat_sim_dcor = pairwise_dcor(dat_sim)
         dat_sim_dcor_diss = squareform(np.round((1- dat_sim_dcor)/2,6))
         dcor_clust = linkage(dat_sim_dcor_diss, method = link_method)
         dcor_clusts = cut_tree(dcor_clust, n_clusters= n_clusts)
         res_mat_out_dim6_dist6[cur_nobs][cur_rep][1] = np.mean(dcor_clusts.flatten() == np.repeat([0,1,2],[3,3,3]))
         
         #w_cor based clustering 
         dat_sim_w_cor = wrapped_covariance_correlation(dat_sim)[1]
         dat_sim_w_cor_diss = squareform(np.round((1- dat_sim_w_cor)/2,6))
         w_cor_clust = linkage(dat_sim_w_cor_diss, method =link_method )
         w_cor_clusts = cut_tree(w_cor_clust, n_clusters= n_clusts)
         res_mat_out_dim6_dist6[cur_nobs][cur_rep][2] = np.mean(w_cor_clusts.flatten() == np.repeat([0,1,2],[3,3,3]))

         #w_dcor based clustering 
         dat_sim_w_dcor = wrapped_dcor(dat_sim)
         dat_sim_w_dcor_diss = squareform(np.round((1- dat_sim_w_dcor)/2,6))
         w_dcor_clust = linkage(dat_sim_w_dcor_diss, method =link_method )
         w_dcor_clusts = cut_tree(w_dcor_clust, n_clusters= n_clusts)
         res_mat_out_dim6_dist6[cur_nobs][cur_rep][3] = np.mean(w_dcor_clusts.flatten() == np.repeat([0,1,2],[3,3,3]))


In [ ]:
print( "n = 50: ", np.mean(res_mat_out_dim6_dist6[0], axis = 0))
print( "n = 100: ", np.mean(res_mat_out_dim6_dist6[1], axis = 0))
print( "n = 500: ", np.mean(res_mat_out_dim6_dist6[2], axis = 0))
print( "n = 1000: ", np.mean(res_mat_out_dim6_dist6[3], axis = 0))

#### Simulations with outliers in the 9 variables

In [ ]:
out_dim = 9
res_mat_out_dim9_dist6= np.ones((len(n_obs_vec),n_reps, 4 ))

In [ ]:
 for cur_nobs in range(len(n_obs_vec)): 
     n_obs = n_obs_vec[cur_nobs]
     #print("current n_obs is ", n_obs)
     for cur_rep in range(n_reps):
         np.random.seed(cur_rep)
         #print('current rep', cur_rep+1)
         
         sd_white_noise = sqrt(0.01)
        
         
         x_1 = np.random.uniform(0,1,n_obs)
         x_1 = -1 + x_1*2
        
         u =   np.random.uniform(0,1,n_obs)
         v =   np.random.uniform(0,1,n_obs) 
         x_4 = np.sqrt(-2 *np.log(u) ) *np.cos(2*pi*v)
         x_7 = np.sqrt(-2 *np.log(u) ) *np.sin(2*pi*v)
         
         x_2 = np.tanh(x_1) + np.power(x_1,2)
         # add outliers to x_2
         out_ind_x_2 = np.random.randint(n_obs, size= round(out_fraction *n_obs) )
         x_2[   out_ind_x_2] = (np.max(x_2) *out_dist)/sqrt(out_dim)
          # add outliers to x_3
         x_3 = 2*np.sin(np.abs(x_1)) + np.random.normal(scale = sd_white_noise,size = n_obs)
         out_ind_x_3 = np.random.randint(n_obs, size= round(out_fraction *n_obs) )
         x_3[out_ind_x_3] = (np.max(x_3) *out_dist)/sqrt(out_dim)
         
         # add outliers to x_5
         x_5 = np.sin(x_4) + np.tanh(x_4) + np.random.normal(scale = sd_white_noise,size = n_obs)
         out_ind_x_5 = np.random.randint(n_obs, size= round(out_fraction *n_obs) )
         x_5[   out_ind_x_5] = (np.max(x_5) *out_dist)/sqrt(out_dim)
         # add outliers to x_6
         x_6 = np.power(x_4,2) + np.random.normal(scale = sd_white_noise,size = n_obs)
         out_ind_x_6 = np.random.randint(n_obs, size= round(out_fraction *n_obs) )
         x_6[out_ind_x_6] = (np.max(x_6) *out_dist)/sqrt(out_dim)
         
         # add outliers to x_8
         x_8 = np.abs(x_7) + np.random.normal(scale = sd_white_noise,size = n_obs)
         out_ind_x_8 = np.random.randint(n_obs, size= round(out_fraction *n_obs) )
         x_8[out_ind_x_8] = (np.max(x_8) *out_dist)/sqrt(out_dim)
         
          # add outliers to x_9
         x_9 = np.sin(np.abs(x_7)) + np.random.normal(scale = sd_white_noise,size = n_obs)
         out_ind_x_9 = np.random.randint(n_obs, size= round(out_fraction *n_obs) )
         x_9[out_ind_x_9] = (np.max(x_9) *out_dist)/sqrt(out_dim)
         
         # add outliers to x_1
         out_ind_x_1 = np.random.randint(n_obs, size= round(out_fraction *n_obs) )
         x_1[out_ind_x_1] = (np.max(x_1) *out_dist)/sqrt(out_dim)
         
         
         # add outliers to x_4
         out_ind_x_4 = np.random.randint(n_obs, size= round(out_fraction *n_obs) )
         x_4[out_ind_x_4] = (np.max(x_4) *out_dist)/sqrt(out_dim)
         
         #add outliers to x_7
         out_ind_x_7 = np.random.randint(n_obs, size= round(out_fraction *n_obs) )
         x_7[out_ind_x_7] = (np.max(x_7) *out_dist)/sqrt(out_dim)
         
         
         dat_sim = np.vstack((x_1,x_2, x_3, x_4, x_5, x_6, x_7, x_8, x_9)).T
         
         #correlation based clustering
         dat_sim_cor = np.corrcoef(dat_sim.T)
         dat_sim_cor_diss = squareform(np.round((1- dat_sim_cor)/2,6))
         cor_clust = linkage(dat_sim_cor_diss, method =link_method )
         cor_clusts = cut_tree(cor_clust, n_clusters= n_clusts)
         res_mat_out_dim9_dist6[cur_nobs][cur_rep][0] = np.mean(cor_clusts.flatten() == np.repeat([0,1,2],[3,3,3]))
         
         #dcor based clustering
         dat_sim_dcor = pairwise_dcor(dat_sim)
         dat_sim_dcor_diss = squareform(np.round((1- dat_sim_dcor)/2,6))
         dcor_clust = linkage(dat_sim_dcor_diss, method = link_method)
         dcor_clusts = cut_tree(dcor_clust, n_clusters= n_clusts)
         res_mat_out_dim9_dist6[cur_nobs][cur_rep][1] = np.mean(dcor_clusts.flatten() == np.repeat([0,1,2],[3,3,3]))
         
         #w_cor based clustering 
         dat_sim_w_cor = wrapped_covariance_correlation(dat_sim)[1]
         dat_sim_w_cor_diss = squareform(np.round((1- dat_sim_w_cor)/2,6))
         w_cor_clust = linkage(dat_sim_w_cor_diss, method =link_method )
         w_cor_clusts = cut_tree(w_cor_clust, n_clusters= n_clusts)
         res_mat_out_dim9_dist6[cur_nobs][cur_rep][2] = np.mean(w_cor_clusts.flatten() == np.repeat([0,1,2],[3,3,3]))

         #w_dcor based clustering 
         dat_sim_w_dcor = wrapped_dcor(dat_sim)
         dat_sim_w_dcor_diss = squareform(np.round((1- dat_sim_w_dcor)/2,6))
         w_dcor_clust = linkage(dat_sim_w_dcor_diss, method =link_method )
         w_dcor_clusts = cut_tree(w_dcor_clust, n_clusters= n_clusts)
         res_mat_out_dim9_dist6[cur_nobs][cur_rep][3] = np.mean(w_dcor_clusts.flatten() == np.repeat([0,1,2],[3,3,3]))


In [ ]:
print( "n = 50: ", np.round(np.mean(res_mat_out_dim9_dist6[0], axis = 0),2))
print( "n = 100: ", np.round(np.mean(res_mat_out_dim9_dist6[1], axis = 0),2))
print( "n = 500: ", np.round(np.mean(res_mat_out_dim9_dist6[2], axis = 0),2))
print( "n = 1000: ", np.round(np.mean(res_mat_out_dim9_dist6[3], axis = 0),2))